In [ ]:
"""
saliency maps by Grad-Cam saliency algorithm from https://github.com/idiap/fullgrad-saliency
"""

'\nsaliency maps by Grad-Cam saliency algorithm from https://github.com/idiap/fullgrad-saliency\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trained_model_path='/content/drive/My Drive/DNCNN_logs'
data_path = '/content/drive/My Drive/data'
dataset = '/Set12'
test_noiseL = 25

In [ ]:
import cv2
import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from skimage.metrics import peak_signal_noise_ratio
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda")

In [ ]:
class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []
        layers.append(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
        layers.append(nn.ReLU(inplace=True))
        for _ in range(num_of_layers-2):
            layers.append(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)
    def forward(self, x):
        out = self.dncnn(x)
        return out

def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        PSNR += peak_signal_noise_ratio(Iclean[i,:,:,:], Img[i,:,:,:], data_range=data_range)
    return (PSNR/Img.shape[0])

In [ ]:
class GradCAMExtractor:
    #Extract tensors needed for Gradcam using hooks
    
    def __init__(self, model):
        self.model = model

        self.features = None
        self.feature_grads = None

        prev_module = None
        self.target_module = None

        # Iterate through layers
        for m in self.model.modules():
            if isinstance(m, nn.Conv2d):
                prev_module = m
            elif isinstance(m, nn.Linear):
                self.target_module = prev_module
                break

        if self.target_module is not None:
            # Register feature-gradient and feature hooks for each layer
            handle_g = self.target_module.register_backward_hook(self._extract_layer_grads)
            handle_f = self.target_module.register_forward_hook(self._extract_layer_features)

    def _extract_layer_grads(self, module, in_grad, out_grad):
        # function to collect the gradient outputs
        self.feature_grads = out_grad[0]
    
    def _extract_layer_features(self, module, input, output):
        # function to collect the layer outputs
        self.features = output

    def getFeaturesAndGrads(self, x, target_class):

        out = self.model(x)

        if target_class is None:
            target_class = out.data.max(1, keepdim=True)[1]

        #output_scalar = -1. * F.nll_loss(out, target_class.flatten(), reduction='sum')
        criterion = nn.MSELoss(size_average=False)
        criterion.cuda()
        noise = torch.FloatTensor(target_class.size()).normal_(mean=0, std=test_noiseL/255.)
        loss = criterion(target_class, noise.cuda()) / (x.size()[0]*2)

        # Compute gradients
        self.model.zero_grad()
        #output_scalar.backward()
        loss.backward()

        return self.features, self.feature_grads


class GradCAM():
    """
    Compute GradCAM 
    """

    def __init__(self, model):
        self.model = model
        self.model_ext = GradCAMExtractor(self.model)


    def saliency(self, image, target_class):
        #Simple FullGrad saliency
        
        self.model.eval()
        features, intermed_grad = self.model_ext.getFeaturesAndGrads(image, target_class=target_class)

        # GradCAM computation
        grads = intermed_grad.mean(dim=(2,3), keepdim=True)
        cam = (F.relu(features)* grads).sum(1, keepdim=True)
        cam_resized = F.interpolate(F.relu(cam), size=image.size(2), mode='bilinear', align_corners=True)
        return cam_resized

In [ ]:
def normalize(data):
    return data/255.

In [ ]:
def save_saliency_map(image, saliency_map, filename):
    """
    save saliency map on image
    Image: tensor of size (3, H, W)
    saliency_map: Tensor of size (1, H, W)
    filename: string with complete path and file extension
    """
    image = image.data.cpu().numpy()
    saliency_map = saliency_map.data.cpu().numpy()

    saliency_map = saliency_map - saliency_map.min()
    saliency_map = saliency_map / saliency_map.max()
    saliency_map = saliency_map.clip(0, 1)

    saliency_map = np.uint8(saliency_map * 255).transpose(1, 2, 0)
    saliency_map = cv2.resize(saliency_map, (224, 224))

    image = np.uint8(image * 255).transpose(1, 2, 0)
    image = cv2.resize(image, (224, 224))

    # Apply JET colormap
    color_heatmap = cv2.applyColorMap(saliency_map, cv2.COLORMAP_JET)

    # Combine image with heatmap
    img_with_heatmap = np.float32(color_heatmap) + np.float32(image)
    img_with_heatmap = img_with_heatmap / np.max(img_with_heatmap)

    cv2.imwrite(filename, np.uint8(255 * img_with_heatmap))

In [ ]:
save_path = '/content/drive/My Drive/DNCNN_saliency_maps/grad_cam'

print('Loading model ...\n')
net = DnCNN(channels=1, num_of_layers=17)
device_ids = [0]
model = nn.DataParallel(net, device_ids=device_ids).cuda()
model.load_state_dict(torch.load(os.path.join(trained_model_path + '/DnCNN-S-25', 'net.pth')))
model.eval()
# load data info
print('Loading data info ...\n')
files_source = glob.glob(os.path.join(data_path + dataset, '*.png'))
files_source.sort() #a list with file names


saliency_methods = GradCAM(model)

def compute_saliency_and_save():
    for f in files_source:
      Img = cv2.imread(f)
      Img = normalize(np.float32(Img[:,:,0]))
      Img = np.expand_dims(Img, 0)
      Img = np.expand_dims(Img, 1)
      ISource = torch.Tensor(Img)
      noise = torch.FloatTensor(ISource.size()).normal_(mean=0, std=test_noiseL/255.)
      INoisy = ISource + noise
      ISource, INoisy = Variable(ISource.cuda()), Variable(INoisy.cuda())
      INoisy = INoisy.requires_grad_()

      # Compute saliency maps for the input data
      saliency_map = saliency_methods.saliency(INoisy, (INoisy-model(INoisy)))

      # Save saliency maps
      for i in range(INoisy.size(0)):
        filename = str(i)
        image = INoisy[i]
        save_saliency_map(image, saliency_map[i], filename + '_'  + '.jpg')

compute_saliency_and_save()

Loading model ...

Loading data info ...



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


AttributeError: ignored